In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* Automated Machine Learning(AutoML) has become widely adopted for building, experimenting and productionizing various types of machine learning models across business use-cases.

* There are different open source solutions available and this notebook explores a simple baseline solution for some of them on the Kaggle TPS (September 2021) competition.

# Import Libraries

In [ ]:
## define configuration
PATH_TRAIN = '../input/tabular-playground-series-sep-2021/train.csv'
PATH_TEST = '../input/tabular-playground-series-sep-2021/test.csv'

PATH_H2OAML_SUBMISSION = 'submission.csv'
PATH_Pyc_SUBMISSION = 'submission_pyc.csv'

MAX_MODEL_RUNTIME_MINS = 10
MAX_MODEL_RUNTIME_SECS = MAX_MODEL_RUNTIME_MINS * 60

In [ ]:
## prepare data
import gc
import os
import shutil
import datatable as dt
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

train = dt.fread(PATH_TRAIN)
test = dt.fread(PATH_TEST)

target = train['claim'].to_numpy().ravel()
test_ids = test['id']

del train[:, ['id', 'claim']]
test = test[:, train.names]

In [ ]:
train = dt.fread("/kaggle/input/tabular-playground-series-sep-2021/train.csv")

train.head()

In [ ]:
test.head()

In [ ]:
## import packages
import h2o
from h2o.automl import H2OAutoML

In [ ]:
## prepare data
h2o.init()

h2o_train = h2o.H2OFrame(train.to_pandas())
h2o_test = h2o.H2OFrame(test.to_pandas())

h2o_train['target'] = h2o.H2OFrame(target).asfactor()

In [ ]:
## run model
features = [x for x in h2o_train.columns if x != 'target']

model_h2o = H2OAutoML(stopping_metric='AUC', max_runtime_secs=MAX_MODEL_RUNTIME_SECS)
model_h2o.train(x=features, y='target', training_frame=h2o_train)

In [ ]:
## check leaderboard
model_h2o.leaderboard

In [ ]:
## generate predictions
preds_h2o = model_h2o.leader.predict(h2o_test).as_data_frame()['True']

In [ ]:
## create submission
submission = dt.Frame(id=test_ids, claim=dt.Frame(preds_h2o))
submission.head()

In [ ]:
## save submission
submission.to_csv("submission.csv")